<a href="https://colab.research.google.com/github/dimasprd/College-Final-Project/blob/main/001_AI_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install segypy
import segypy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, max_error, mean_squared_error, mean_pinball_loss

import warnings

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 200
plt.style.use('seaborn')

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


# Load the Dataset

In [3]:
dataset_F032 = pd.read_excel('/content/drive/My Drive/Final Project/Well-F032.xlsx')
dataset_F021 = pd.read_excel('/content/drive/My Drive/Final Project/Well-F021.xlsx')
dataset_F061 = pd.read_excel('/content/drive/My Drive/Final Project/Well-F061.xlsx')